# IaaS and IaaC

Infrastructure   
1. Compute consists of VMS such as EC2, Containers such as Docker
2. Storage consists of Object Storage (S3), Non-relational Stores (DynamoDB), and Relationsl Stores (Amazon RDS, Google Cloud SQL)
3. Networking consists of Virtual Private Cloud (VPC), Load Balancers (AWS ELB)  


As a service model consists of 
1. A web service accessible over HTTP
2. Typically exposing APIs with client side tools including CLIs, SDKs, UI
3. Multi-tenant- same service is able to support multiple users
4. Pay as you use charging model

IaaS- web service that supports resource management   
Provisioning and Management includes Creating, updating, and managing resources. Infrastructure provisioning in the real world is done through tools that support a declarative model for provisioning and management

### AWS CloudFormation

Create a template describing the infrastructure resources you want in a YAML/JSON format (this is a declarative definition)  
Give the template to the CloudFormation web service, which takes care of provisioning all infrastructure resources defined in the template.   
Technically Infrastructure as Code, which is a web service that works with infrastructure defined in this declarative format.  
You express what is required, not how to do it.

### Advantages of "as-Code"

1. Easier to understand what is being provisioned. With a block of code, it's hard to easily see what is being provisioned.
2. Robust provisioning of Infrastructure, if a single resouce fails, it will restart
3. Infrastructure definition templates can be shared with others.
4. Easy to recreate defined infrastructure
5. Easy to track infrastructure resource templates

High level abstractions with Infrastructure resource (Look at first paragraph)  
Target user is person managing architecture, skills needed is understanding of resources and scripting.

### AWS CloudFormation (again)

Abstraction is resources, templates is the formatted file that defined the resources needed, stack is the set of resources provisioned.   

### What is Declared?

Resources with different properties, input/output parameters. IAM permissions are **not declared**

In order to make a template reusable, CloudFormation uses input parameters and mappings. IAM has Policy Variables.  
In order to define ordering between different resources, it is either explicitly specified or implicitly defined (through an attribute).  
All actions are supported (CRUD)  
Operations can be All or nothing (either creates or doesn't, and if some action fails, then you delete all previously created stuff) and partially created

### Resources

Top level object in CloudFormation template:  
Resources: 
HelloBucket (Logical Name)  
Type: AWS::S3::Bucket   
Logical name is the name used in the template, physical name is generated by AWS and is a conbination of Logical name, stack name, and unique ID.  
Resource properties are specified with the resource like such:   
HelloBucket (Logical Name)  
Type: AWS::S3::Bucket  
Properties:   
AccessControl: PublicRead  
In addition, you can have multiple properties, and each property can have multiple values

### Resource Attributes

CreationPolicy: Attribute when resource prevents status from reaching create complete until CloudFormation receives number of success signals or until timeout is exceeded.   
DeletionPolicy: Attribute that allows you to preserve the state of a resource before it is deleted, done through retain policy *DeletionPolicy: Retain*   
DependsOn - attribute that allows you to specify whether you want the creation of an attribute to follow another i.e. *DependsOn: myDB*.  
Metadata- attribute that associated structured data with resource.  
Intrinsic functions provided by AWS, use these in your templates to assign values to properties not available until runtime. Can be used in resource policies, outputs, metadata attributes, update policy attributes   
Ref- in built function that takes in the logical name, outputs the physical name   
GetAtt- gets the attributes of resource, takes in logical name of resource and name of attribute, outputs attribute value.  

Parameter definition consists of type of parameter, contraints on parameter value, default value

# Platform and PaaS

Platform is a set of one or more infrastructure resources configured to run specific apps (i.e. Python web platform, EC2 instance configured with Python2.7 and Nginx)

### PaaS

PaaS is a web service that applies transformations on infrastructure resources to generate the required plaform, also deploy apps on the generated platform,

### IaaS vs PaaS

IaaS mostly just creates the infrastructure, user then has to deal with the application deployment, but PaaS does deployment on top of infrastructure

You can do some of the PaaS stuff with IaaC. IaC could be good for deploying pre-built static apps, but other than that it's not good for deployment. Normally, it's used by a devops person, not for app devs.

IaaC is great for pre-built apps that never change, but if it is under constant change, IaC does not have the ability to build application from source code.

Using PaaS, app developers can do production applicatino deployment. This has led to DevOps engineers

### DevOps

Interpretation 1: It is a functional role in which they occasionally perform Operations duties.  
Interpretation 2: It is a set of steps/processes, and these engineers use development practices when setting up infrastructure.

App developers need to be aware of infrastructure, and should not treat it as a black box. Actual infrastructure work is still done by Operations Engineers.   
Operations Engineers need to practice good development practices and closely work with app devs to determine production architecture and required infrastructure

### PaaS features

Allows you to build/deploy applications, provides a load balancer, DNS name connected to load balancer, ability to configure SSL Certs, scales application (manually or triggered by policies), canary deployment, and deployment without any downtime, rollbacks, and log collection

# Web apps

Web pages - static. 
Web applications - dynamic   
Web sites - web pages + web applications.  
Primary characteristics are dynamic content, persistent storage, API access. Uses HTTP, frameworks to make it, and is logically divided into three layers (HTML UI, Web App request handling, server logic, data access layer to DB)

# EBS

Amazon's PaaS, deploys web apps through source code to EC2, and beanstalk handles EC2 provisioning.   
Application can use other AWS resources, but Beanstalk provisions a LoadBalancer for the deployed application and the app can provision a routable DNS record

1. Application: OS folder that contains the app source code.  
2. Application Version: Specific version of the application code, stored on S3
3. Environment: Application version deployed on a set of AWS resources, each environment runs a single application version
4. Environment Tier: Tier determines whether the resources provisioned by Beanstalk would be used for deploying a backend or a web app that handles HTTP traffic
5. Environment Configuration: determines prameters and settings of environment's resources

### Host Manager

Beanstalk software that runs on each EC2 instance in an environment, has to deploy app, aggregate events, generate events, monitor logs, monitor app server, patch instance components, publish logs to S3

Environment types:
1. Load balancing/autoscaling
2. Single Instance

### IAM Permissions

User policies: permissions that your IAM user needs to interact with EBS.   
Service Role: Needed by EBS to interact with other AWS services on your behalf.  
Instance Profile: Role and permissions granted to EC2 instance in your environment

### User Policy

EBS will use the permissions given to the IAM user when creating other AWS resources, user needs FullAccess managed Policy, EBS uses user permissions to launch all resources in environment

### Service Role

Contains all nevessary permissions that allows EBS to call other AWS services without user intervention, permissions that allow EBS to monitor instance and environment health and update nevironments to perform updates. A service role is created by default when creating an environment

### Instance Profile

AWSElasticBeanstalkWebTier - Policy Statement that allows EC2 instantces in your environment to upload logs to S3, default instance profile is aws-elasticbeanstalk-ec2-role

### Platform/Container type

Software stack to run on EC2 instances provisioned as part of an environment is determined by the platform/container type of environment. This is different from Docker. We're talking about Apache Tomcat here.

### PaaS criticisms

1. Not able to change software stack used by application. This is changing with Docker
2. Less control, not possible to control configuration and setting of IaaS resources. 
3. Once you use a PaaS on one vendor, it's hard to migrate to other PaaS
4. It's costly and difficult to use when apps are being developed

All AWS resources are provisioned as a part of an environment that can be customized using a config file in the app folder. Things that can be customized  include the software stack settings on the EC2 instances, type and flavor of provisioned resources.   
.ebextensions includes files with config files. 
Use services to define the services/processes you want started on your EC2 instances. Container commands are run after the app/web server have been set up and the application version has been extracted.   
You can use CloudFormation snippets to customize AWS resources. This is unique to EBS. EBS can also create an RDS instance, and EBS will provide the data in environment variables that needs to be read in.

# Docker

Docker solves the problem that is it should be possible to repeatedly deploy an application whilst always getting the same result.   
This is important, because any app that is developed and tested locally should work exactly like that in production. Applications should behave the same across different environments

### Repeatability Issues

1. It's hard to ensure that there's similarity between environment components across environments. This is usually solved by PaaS or IaC for deployment
2. Consistency of packages and libraries across different environments is hard. This has been solved by packages, but OS-level packages is harder, so the way you do it is by packaging OS along with the app

Challenges with packaging OS is it's hard to define what OS we want, what the end result of the packaging would look like, where the App + OS would run, how to bind it to databases, and how to access the app

### Utilization Issues

Modern Hosts have large capacity, so containers allow us to maximize the utilization of a host

### Resource Isolation

We need to isolate the Dockerized OS with the host OS. Requirements are:   
1. Root file systems of Containers should remain separate
2. Processes that are running inside Containers should remain separate
3. Network traffic destined for different Containers should remain separate
4. A rouge container should not be able to affect other containers running on the host

### Docker

Docker is a technology for packaging OS + Application.   
It started as a technologu differentiator within PaaS called DotCloud.   
Core Docker technology open sourced in 2013, renamed to Docker Inc.

### Dockerfile

Declarative definition format that defined Base OS, any packages, where to inject app, what environment variable to set, which port the app will listen to

### How does Docker work?

Docker daemon pulls down base OS image specified in the Dockerfile, which is located in the Dockerfile in the root directory.  
Docker daemon executes each line defined in the Dockerfile on this base image. This daemon builds the final image in the form of layers, and each line in the Dockerfile causes a new diff layer to be generated over the previous layer

### Basic Docker Commands

Build a docker container is `docker build -t <tag-name>`. Dockerfile needs to be present in the directory from which this command is executed   
Execute a docker container is `docker run -d -p <host-port>:<container-port><tag-name>` -d runs Docker in detached mode, -p maps container port to host port, tag-name is the name used in the `docker build` command.   
To see logs, you run `docker logs <container-id>`.   
To see all running Docker containers `docker ps`.  
To stop a docker container `docker stop <container-id>`   
To remove a stopped Docker container `docker rm <container-id>`.   
To remove a docker container image run `docker rmi <tag-name>`